In [10]:
import os
import pandas as pd

# Путь к папке с CSV файлами
base_dir = "CIC-IDS-2017/csvs/"

# Получение списка папок атак
attack_types = os.listdir(base_dir)

# Список атак, которые нужно исключить
excluded_attacks = ["sql-inj"]

# Фильтрация attack_types для удаления исключённых атак
attack_types = [attack for attack in attack_types if attack not in excluded_attacks]


# Функция для обработки данных одной атаки
def process_attack_data(attack_folder):
    attack_path = os.path.join(base_dir, attack_folder)

    # Списки для нормального и атакующего трафика
    normal_files = os.listdir(os.path.join(attack_path, "normal"))
    attack_files = os.listdir(os.path.join(attack_path, "attack"))

    # Словарь для хранения корреляций по пакетным интервалам
    correlation_data = {}

    # Для каждого интервала пакетных выборок
    for normal_file in normal_files:
        # Определяем соответствующий файл для атаки
        packet_size = normal_file.split("_")[1]
        attack_file = [f for f in attack_files if f"_{packet_size}_" in f]

        if attack_file:
            # Чтение нормального и атакующего трафика
            normal_df = pd.read_csv(os.path.join(attack_path, "normal", normal_file))
            attack_df = pd.read_csv(os.path.join(attack_path, "attack", attack_file[0]))

            # Объединяем нормальные и атакующие данные
            combined_df = pd.concat([normal_df, attack_df])

            # Вычисление корреляционной матрицы
            corr_matrix = combined_df.corr()

            # Выделение корреляции с меткой "Label"
            target_corr = corr_matrix["Label"].abs().sort_values(ascending=False)

            # Сохраняем топ-5 коррелирующих признаков (исключая 'Label')
            top_features = target_corr.index[1:]  # 5 наиболее значимых признаков

            # Сохраняем корреляции для текущего интервала
            correlation_data[packet_size] = target_corr[top_features]

    # Преобразуем словарь в DataFrame, строки - это интервал пакетов, столбцы - признаки
    correlation_df = pd.DataFrame(
        correlation_data
    ).T  # Транспонируем для правильной формы

    # Преобразуем индекс из строк в числа для корректной сортировки
    correlation_df.index = correlation_df.index.astype(int)

    # Добавляем столбец с количеством признаков, имеющих корреляцию больше 0.7
    correlation_df['High Correlation Count'] = (correlation_df > 0.7).sum(axis=1)

    # Округление значений до 3 знаков после запятой
    correlation_df = correlation_df.round(3)

    # Сортировка по индексу (длине пакетной выборки)
    correlation_df = correlation_df.sort_index()

    # Вывод таблицы на экран
    print(f"Correlation table for attack type: {attack_folder}")
    print(correlation_df.to_string(index=True))

    # Сохраняем DataFrame в CSV файл
    correlation_df.to_csv(
        f"{attack_folder}_correlation_with_label.csv", index_label="Packet Interval"
    )


# Проход по каждой атаке
for attack in attack_types:
    process_attack_data(attack)

Correlation table for attack type: brute
       Average Packet Size  Flow Bytes/s  Flow IAT Mean  Fwd Header Length  Fwd IAT Min  Fwd IAT Std  Fwd Packet Length Max  Fwd Packet Length Mean  Max Packet Length  Total Length of Fwd Packets  High Correlation Count
10                   0.176         0.161          0.122              0.316        0.350        0.113                  0.268                   0.272              0.176                        0.176                       0
50                   0.353         0.166          0.316              0.357        0.339        0.273                  0.187                   0.182              0.353                        0.353                       0
100                  0.473         0.139          0.464              0.388        0.224        0.398                  0.336                   0.330              0.473                        0.473                       0
250                  0.576         0.005          0.641              0.476     

In [12]:
import os
import pandas as pd

# Путь к папке с CSV файлами
base_dir = "CIC-IDS-2017/csvs/"

# Получение списка папок атак
attack_types = os.listdir(base_dir)

# Список атак, которые нужно исключить
excluded_attacks = ["sql-inj"]

# Фильтрация attack_types для удаления исключённых атак
attack_types = [attack for attack in attack_types if attack not in excluded_attacks]


# Функция для обработки данных одной атаки
def process_attack_data(attack_folder):
    attack_path = os.path.join(base_dir, attack_folder)

    # Списки для нормального и атакующего трафика
    normal_files = os.listdir(os.path.join(attack_path, "normal"))
    attack_files = os.listdir(os.path.join(attack_path, "attack"))

    # Словарь для хранения корреляций по пакетным интервалам
    correlation_data = {}

    # Для каждого интервала пакетных выборок
    for normal_file in normal_files:
        # Определяем соответствующий файл для атаки
        packet_size = normal_file.split("_")[1]
        attack_file = [f for f in attack_files if f"_{packet_size}_" in f]

        if attack_file:
            # Чтение нормального и атакующего трафика
            normal_df = pd.read_csv(os.path.join(attack_path, "normal", normal_file))
            attack_df = pd.read_csv(os.path.join(attack_path, "attack", attack_file[0]))

            # Объединяем нормальные и атакующие данные
            combined_df = pd.concat([normal_df, attack_df])

            # Вычисление корреляционной матрицы
            corr_matrix = combined_df.corr(method="spearman")

            # Выделение корреляции с меткой "Label"
            target_corr = corr_matrix["Label"].abs().sort_values(ascending=False)

            # Сохраняем топ-5 коррелирующих признаков (исключая 'Label')
            top_features = target_corr.index[1:]  # 5 наиболее значимых признаков

            # Сохраняем корреляции для текущего интервала
            correlation_data[packet_size] = target_corr[top_features]

    # Преобразуем словарь в DataFrame, строки - это интервал пакетов, столбцы - признаки
    correlation_df = pd.DataFrame(
        correlation_data
    ).T  # Транспонируем для правильной формы

    # Преобразуем индекс из строк в числа для корректной сортировки
    correlation_df.index = correlation_df.index.astype(int)

    # Добавляем столбец с количеством признаков, имеющих корреляцию больше 0.7
    correlation_df["High Correlation Count"] = (correlation_df > 0.7).sum(axis=1)

    # Округление значений до 3 знаков после запятой
    correlation_df = correlation_df.round(3)

    # Сортировка по индексу (длине пакетной выборки)
    correlation_df = correlation_df.sort_index()

    # Вывод таблицы на экран
    print(f"Correlation table for attack type: {attack_folder}")
    print(correlation_df.to_string(index=True))

    # Сохраняем DataFrame в CSV файл
    correlation_df.to_csv(
        f"{attack_folder}_spearman_correlation_with_label.csv",
        index_label="Packet Interval",
    )


# Проход по каждой атаке
for attack in attack_types:
    process_attack_data(attack)

Correlation table for attack type: brute
       Average Packet Size  Flow Bytes/s  Flow IAT Mean  Fwd Header Length  Fwd IAT Min  Fwd IAT Std  Fwd Packet Length Max  Fwd Packet Length Mean  Max Packet Length  Total Length of Fwd Packets  High Correlation Count
10                   0.338         0.380          0.408              0.303        0.562        0.424                  0.378                   0.378              0.340                        0.340                       0
50                   0.336         0.127          0.162              0.383        0.537        0.197                  0.150                   0.152              0.335                        0.335                       0
100                  0.427         0.130          0.206              0.447        0.508        0.067                  0.007                   0.001              0.430                        0.430                       0
250                  0.538         0.409          0.576              0.508     